## Write all Profiles to GCT for Heatmap Visualization

**Gregory Way, 2020**

I also build consensus signatures for all unique treatments and output associated files.

In [1]:
import os
import pandas as pd

from pycytominer import feature_select
from pycytominer.cyto_utils import infer_cp_features, write_gct

from scripts.processing_utils import load_data

In [2]:
# Set constants
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "drop_na_columns",
    "blocklist",
    "drop_outliers"
]

gct_dir = os.path.join("data", "gct_files")
profile_dir = os.path.join("..", "0.generate-profiles", "profiles")
cell_count_dir = os.path.join("..", "0.generate-profiles", "cell_counts")
output_dir = os.path.join("data", "merged")

suffix = "normalized.csv.gz"

# Ignore 40X batch
batches = [x for x in os.listdir(profile_dir) if x not in [".DS_Store", "2019_02_15_Batch1_40X"]]
batches

['2021_03_03_Batch14',
 '2021_03_03_Batch13',
 '2021_03_03_Batch12',
 '2021_03_03_Batch15',
 '2020_08_24_Batch9',
 '2021_03_05_Batch17',
 '2021_03_05_Batch16',
 '2021_02_08_Batch11',
 '2021_08_03_Batch26',
 '2021_07_21_Batch22',
 '2021_07_21_Batch23',
 '2021_08_03_Batch27',
 '2019_03_20_Batch2',
 '2019_06_25_Batch3',
 '2020_09_08_Batch10',
 '2020_07_02_Batch8',
 '2019_11_11_Batch4',
 '2021_03_12_Batch18',
 '2021_06_25_Batch21',
 '2021_03_12_Batch19',
 '2021_06_25_Batch20',
 '2019_11_20_Batch6',
 '2019_02_15_Batch1_20X',
 '2019_11_19_Batch5',
 '2021_08_02_Batch24',
 '2019_11_22_Batch7',
 '2021_08_02_Batch25']

In [3]:
profile_batches = {}
for batch in batches:
    # Build output information
    output_gct_dir = os.path.join(gct_dir, batch)
    os.makedirs(output_gct_dir, exist_ok=True)
    output_gct_file = os.path.join(
        output_gct_dir, "{}_feature_select.gct".format(batch)
    )
    
    # Load the profile data and add cell counts
    df = load_data(
        batch=batch,
        suffix=suffix,
        profile_dir=profile_dir,
        combine_dfs=True,
        add_cell_count=True,
        harmonize_cols=True,
        cell_count_dir=cell_count_dir
    )

    # Save normalized and non-feature selected data
    profile_batches[batch] = df
    
    # Apply feature selection
    feature_select_df = feature_select(df, operation=feature_select_ops)
        
    # Write the dataframe as a gct file for input into Morpheus
    write_gct(profiles=feature_select_df, output_file=output_gct_file)

## Merge Profiles Together and Output

In [4]:
all_profiles_df = pd.concat(profile_batches.values(), sort=True).reset_index(drop=True)

all_profiles_df = all_profiles_df.assign(Metadata_clone_type="resistant")
all_profiles_df.loc[all_profiles_df.Metadata_clone_number.str.contains("WT"), "Metadata_clone_type"] = "wildtype"

meta_features = infer_cp_features(all_profiles_df, metadata=True)
cp_cols = infer_cp_features(all_profiles_df, metadata=False)

all_profiles_df = all_profiles_df.reindex(meta_features + cp_cols, axis="columns")

print(all_profiles_df.shape)
all_profiles_df.head()

(3957, 3544)


,Metadata_Plate,Metadata_Well,Metadata_batch,Metadata_cell_count,Metadata_cell_density,Metadata_celltype_shorthand_from_plate_graph,Metadata_clone_number,Metadata_date,Metadata_plate_ID,Metadata_plate_filename,...,Nuclei_Texture_Variance_RNA_10_02,Nuclei_Texture_Variance_RNA_10_03,Nuclei_Texture_Variance_RNA_20_00,Nuclei_Texture_Variance_RNA_20_01,Nuclei_Texture_Variance_RNA_20_02,Nuclei_Texture_Variance_RNA_20_03,Nuclei_Texture_Variance_RNA_5_00,Nuclei_Texture_Variance_RNA_5_01,Nuclei_Texture_Variance_RNA_5_02,Nuclei_Texture_Variance_RNA_5_03
0,219905,B02,2021_03_03_Batch14,4269,2.5x10^3 cells/well,1.0,WT_parental,20210219.0,NaN,NaN,...,-1.007074,-1.010473,-1.018616,-1.038237,-1.005599,-1.015286,-1.020732,-1.009446,-1.019474,-1.012719
1,219905,B03,2021_03_03_Batch14,1688,2.5x10^3 cells/well,2.0,CloneA,20210219.0,NaN,NaN,...,-0.986309,-1.000248,-1.022190,-1.045947,-1.016479,-1.038013,-0.999631,-0.999447,-0.987759,-0.977462
2,219905,B04,2021_03_03_Batch14,2238,2.5x10^3 cells/well,3.0,CloneE,20210219.0,NaN,NaN,...,-1.371187,-1.358646,-1.353107,-1.355997,-1.357253,-1.352782,-1.360769,-1.364501,-1.366883,-1.360609
3,219905,B05,2021_03_03_Batch14,3358,2.5x10^3 cells/well,4.0,WT clone 01,20210219.0,NaN,NaN,...,-1.454741,-1.437857,-1.441065,-1.458476,-1.441738,-1.481096,-1.456043,-1.463368,-1.462129,-1.456112
4,219905,B06,2021_03_03_Batch14,291,2.5x10^3 cells/well,5.0,WT clone 02,20210219.0,NaN,NaN,...,0.334975,0.403553,0.461756,0.434181,0.315446,0.586550,0.342533,0.354969,0.379586,0.361450


In [5]:
all_profiles_df = feature_select(all_profiles_df, operation=feature_select_ops)

all_profiles_df = all_profiles_df.drop(["Metadata_plate_ID", "Metadata_plate_filename"], axis="columns")

print(all_profiles_df.shape)
all_profiles_df.head()

(3957, 458)


,Metadata_Plate,Metadata_Well,Metadata_batch,Metadata_cell_count,Metadata_cell_density,Metadata_celltype_shorthand_from_plate_graph,Metadata_clone_number,Metadata_date,Metadata_plate_map_name,Metadata_time_to_adhere,...,Nuclei_Texture_Correlation_RNA_20_00,Nuclei_Texture_Correlation_RNA_20_02,Nuclei_Texture_Entropy_DNA_20_03,Nuclei_Texture_Entropy_ER_20_03,Nuclei_Texture_InfoMeas1_DNA_5_02,Nuclei_Texture_InfoMeas1_ER_5_00,Nuclei_Texture_InfoMeas1_Mito_5_00,Nuclei_Texture_InfoMeas1_RNA_5_00,Nuclei_Texture_SumAverage_DNA_20_03,Nuclei_Texture_SumVariance_AGP_20_03
0,219905,B02,2021_03_03_Batch14,4269,2.5x10^3 cells/well,1.0,WT_parental,20210219.0,219905,48 hr,...,-0.393503,-0.656457,-1.525862,-0.689886,1.381230,1.683708,0.890482,1.819708,-0.654685,-1.201057
1,219905,B03,2021_03_03_Batch14,1688,2.5x10^3 cells/well,2.0,CloneA,20210219.0,219905,48 hr,...,1.202557,1.040639,-0.737619,-0.873918,0.133264,-0.753028,-0.729984,-0.761175,-1.688990,-0.894227
2,219905,B04,2021_03_03_Batch14,2238,2.5x10^3 cells/well,3.0,CloneE,20210219.0,219905,48 hr,...,1.782753,1.151987,-1.144326,-2.081855,0.613646,0.124588,0.677733,-0.322159,-1.776875,-1.199676
3,219905,B05,2021_03_03_Batch14,3358,2.5x10^3 cells/well,4.0,WT clone 01,20210219.0,219905,48 hr,...,-0.730221,-0.534683,-1.467039,-1.865624,1.895769,1.285182,0.735492,1.617357,-1.205825,-1.531649
4,219905,B06,2021_03_03_Batch14,291,2.5x10^3 cells/well,5.0,WT clone 02,20210219.0,219905,48 hr,...,0.727340,1.228002,1.874837,0.763804,-1.437746,-0.150566,-0.905429,-0.617372,1.571506,0.849895


In [6]:
output_file = os.path.join(output_dir, "all_merged_profiles.csv.gz")
all_profiles_df.to_csv(output_file, index=False, compression="gzip")

output_gct_file = os.path.join(gct_dir, "all_merged_profiles.gct")
write_gct(profiles=all_profiles_df, output_file=output_gct_file)

## Collapse replicates into consensus profiles

In [7]:
median_consensus_df = (
    all_profiles_df.groupby(["Metadata_clone_number", "Metadata_treatment"])
    .median()
    .reset_index()
)

print(median_consensus_df.shape)
median_consensus_df.head()

(186, 450)


,Metadata_clone_number,Metadata_treatment,Metadata_cell_count,Metadata_celltype_shorthand_from_plate_graph,Metadata_date,Metadata_treatment_shorthand_from_plate_graph,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_FormFactor,Cells_AreaShape_MedianRadius,...,Nuclei_Texture_Correlation_RNA_20_00,Nuclei_Texture_Correlation_RNA_20_02,Nuclei_Texture_Entropy_DNA_20_03,Nuclei_Texture_Entropy_ER_20_03,Nuclei_Texture_InfoMeas1_DNA_5_02,Nuclei_Texture_InfoMeas1_ER_5_00,Nuclei_Texture_InfoMeas1_Mito_5_00,Nuclei_Texture_InfoMeas1_RNA_5_00,Nuclei_Texture_SumAverage_DNA_20_03,Nuclei_Texture_SumVariance_AGP_20_03
0,BZ001,0.1% DMSO,2526.0,9.0,20210226.0,1.0,-0.504836,0.012977,1.029371,-0.108956,...,-0.725983,-0.727778,0.112075,0.723267,-0.582920,-1.182867,-0.875848,-0.886125,-0.037109,0.797380
1,BZ001,0.78 nM bortezomib,955.5,9.0,20210265.0,2.0,-0.734515,-0.022992,0.932717,0.033168,...,-0.667964,-0.993440,-0.171474,0.678733,-0.440669,-0.677444,-0.142691,-0.235062,-0.031662,0.724883
2,BZ001,2.33 nM bortezomib,978.5,9.0,20210265.0,2.0,-0.853896,-0.461451,0.775808,-0.075467,...,-0.801871,-0.951312,-0.809575,0.398809,-0.042569,-0.586292,-0.014746,0.231434,-0.111685,0.631403
3,BZ001,21 nM bortezomib,24448.0,NaN,NaN,NaN,0.163763,0.030651,0.257632,-0.095946,...,-0.568693,-0.716287,0.254292,-0.148769,0.189309,0.170067,0.360550,0.324726,0.586299,-0.338752
4,BZ001,7 nM bortezomib,1061.5,9.0,20210265.0,2.0,-0.735779,-0.484946,0.750878,-0.486038,...,-0.956341,-1.147708,-0.585893,0.445845,-0.223588,-0.838398,-0.041782,-0.314503,-0.139073,0.809276


In [8]:
output_file = os.path.join(output_dir, "consensus_profiles.csv.gz")
median_consensus_df.to_csv(output_file, index=False, compression="gzip")

output_gct_file = os.path.join(gct_dir, "consensus_profiles.gct")
write_gct(profiles=median_consensus_df, output_file=output_gct_file)